## Importação das bibliotecas

In [144]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

In [145]:
data = pd.read_csv('Heart Failure Clinical Records.csv')
data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Análise exploratória

In [146]:
print('Tamanho: ', end="")
print(data.shape)
print('-----------------------------------------------------------------------------------------------------------')
display(data.describe())
print('-----------------------------------------------------------------------------------------------------------')
print(data.info())

Tamanho: (299, 13)
-----------------------------------------------------------------------------------------------------------


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


-----------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT 

Aqui observamos que temos mais mulheres do que homens na nossa base de dados.

In [147]:
tmp = data

tmp['sex'] = tmp['sex'].apply(lambda x: 'Mulher' if x == 0 else 'Homem')
tmp['DEATH_EVENT'] = tmp['DEATH_EVENT'].apply(lambda x: 'Não faleceu' if x == 0 else 'Faleceu')

In [ ]:
fig = px.pie(tmp, names='sex', width=500, color_discrete_sequence=['#19647E', '#A31621'])
fig.update_xaxes(title_text='Sexo')
fig.update_yaxes(title_text='')
fig.update_layout(bargap=0.2)
fig.show()

Aqui podemos ver que apenas 32% dos homens e 32,4% das mulheres da base de dados faleceram durante o acompanhamento

In [161]:
fig = px.histogram(tmp, x='DEATH_EVENT', width=500, 
             color_discrete_sequence=['#19647E', '#A31621'], color='sex', labels={"sex": "Sexo"})
fig.update_xaxes(title_text='Evento')
fig.update_yaxes(title_text='')
fig.update_layout(bargap=0.2)
fig.show()

Aqui podemos observar que, dos pacientes falecidos, os homens são os que mais acabaram morrendo durante o acompanhamento

In [151]:
graph_df = data.loc[tmp['DEATH_EVENT'] == 'Faleceu']
fig = px.pie(graph_df, names='sex', width=500, 
             color_discrete_sequence=['#19647E', '#A31621'])
fig.show()